# Ensemble models

## Hippocampus

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, randint
from scipy import linalg

# Load and preprocess data
def load_and_preprocess_data():
    densenet_df = pd.read_csv('densenet_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')
    pointnet_df = pd.read_csv('pointnet_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')
    random_forest_df = pd.read_csv('random_forest_pyradiomics_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')

    X = np.hstack((
        densenet_df.iloc[:, 2:17].values,
        pointnet_df.iloc[:, 2:17].values,
        random_forest_df.iloc[:, 2:17].values
    ))
    y = densenet_df['Actual'].values

    return X, y, densenet_df, pointnet_df, random_forest_df

# Ensemble methods
def simple_average_ensemble(X):
    return np.mean(X, axis=1)

def weighted_average_ensemble(X_train, y_train, X_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    return lr_model.predict(X_test)

def non_linear_weighted_average(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def tune_rf_meta_learner(X_train, y_train):
    param_dist = {
        'n_estimators': randint(50, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    }
    rf = RandomForestRegressor(random_state=42)
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)
    rf_random.fit(X_train, y_train)
    return rf_random.best_estimator_

def stacking_ensemble(X_train, y_train, X_test):
    models = [
        RandomForestRegressor(n_estimators=100, random_state=42),
        GradientBoostingRegressor(n_estimators=100, random_state=42),
        SVR(kernel='rbf')
    ]
    
    second_level_train = np.column_stack([
        cross_val_predict(model, X_train, y_train, cv=5) 
        for model in models
    ])
    second_level_test = np.column_stack([
        model.fit(X_train, y_train).predict(X_test) 
        for model in models
    ])
    
    # Use Ridge regression as the meta-learner
    alpha = 1.0
    n_samples, n_features = second_level_train.shape
    
    # Add a small value to the diagonal for numerical stability
    A = np.dot(second_level_train.T, second_level_train)
    A.flat[::n_features + 1] += alpha + 1e-10
    
    b = np.dot(second_level_train.T, y_train)
    
    # Solve the linear system
    try:
        coef = linalg.solve(A, b, assume_a='pos')
    except TypeError:
        # For older versions of SciPy
        coef = linalg.solve(A, b, sym_pos=True)
    
    # Make predictions
    return np.dot(second_level_test, coef)

# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    correlation, _ = pearsonr(y_true, y_pred)
    return mse, correlation

# Main execution
def main():
    X, y, densenet_df, pointnet_df, random_forest_df = load_and_preprocess_data()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    simple_avg_scores = []
    weighted_avg_scores = []
    non_linear_weighted_scores = []
    rf_meta_scores = []
    stacking_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Simple average
        simple_avg_pred = simple_average_ensemble(X_test)
        simple_avg_scores.append(evaluate_model(y_test, simple_avg_pred))
        
        # Weighted average
        weighted_avg_pred = weighted_average_ensemble(X_train, y_train, X_test)
        weighted_avg_scores.append(evaluate_model(y_test, weighted_avg_pred))
        
        # Non-linear weighted average
        non_linear_weighted_pred = non_linear_weighted_average(X_train, y_train, X_test)
        non_linear_weighted_scores.append(evaluate_model(y_test, non_linear_weighted_pred))
        
        # Random Forest meta-learner (with hyperparameter tuning)
        rf_model = tune_rf_meta_learner(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_meta_scores.append(evaluate_model(y_test, rf_pred))
        
        # Stacking ensemble
        stacking_pred = stacking_ensemble(X_train, y_train, X_test)
        stacking_scores.append(evaluate_model(y_test, stacking_pred))

    # Calculate and print average scores
    print("Ensemble Methods:")
    print(f"Simple Average Ensemble - MSE: {np.mean([s[0] for s in simple_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in simple_avg_scores]):.4f}")
    print(f"Weighted Average Ensemble - MSE: {np.mean([s[0] for s in weighted_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in weighted_avg_scores]):.4f}")
    print(f"Non-linear Weighted Average - MSE: {np.mean([s[0] for s in non_linear_weighted_scores]):.4f}, Correlation: {np.mean([s[1] for s in non_linear_weighted_scores]):.4f}")
    print(f"Tuned Random Forest Meta-learner - MSE: {np.mean([s[0] for s in rf_meta_scores]):.4f}, Correlation: {np.mean([s[1] for s in rf_meta_scores]):.4f}")
    print(f"Stacking Ensemble - MSE: {np.mean([s[0] for s in stacking_scores]):.4f}, Correlation: {np.mean([s[1] for s in stacking_scores]):.4f}")

    # Evaluate original models
    print("\nOriginal Models:")
    for model_name, df in zip(['DenseNet', 'PointNet', 'Random Forest'], 
                              [densenet_df, pointnet_df, random_forest_df]):
        avg_pred = df['Average_Prediction'].values
        mse, correlation = evaluate_model(df['Actual'], avg_pred)
        print(f"{model_name} - MSE: {mse:.4f}, Correlation: {correlation:.4f}")

if __name__ == "__main__":
    main()

Ensemble Methods:
Simple Average Ensemble - MSE: 0.0707, Correlation: 0.1623
Weighted Average Ensemble - MSE: 0.1992, Correlation: 0.0777
Non-linear Weighted Average - MSE: 0.1021, Correlation: 0.1704
Tuned Random Forest Meta-learner - MSE: 0.0844, Correlation: -0.0082
Stacking Ensemble - MSE: 0.0816, Correlation: 0.0310

Original Models:
DenseNet - MSE: 0.0753, Correlation: 0.1425
PointNet - MSE: 0.0779, Correlation: 0.0036
Random Forest - MSE: 0.0700, Correlation: 0.1790


### MLP with hyperparameters

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    # print(f"Config: {config}")
    # print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    # print("-" * 50)

# Sort results by MSE (ascending) and positive correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: x[2], reverse=True)

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Correlation
print_top_5(results_by_corr, "Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")

# Diagnostic: Print predictions vs actual for best correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")



Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0929
   Correlation: 0.1766

2. Configuration:
   hidden_sizes: [128]
   dropout_rate: 0.5
   lr: 0.0005
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0958
   Correlation: 0.2266

3. Configuration:
   hidden_sizes: [64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1027
   Correlation: 0.1901

4. Configuration:
   hidden_sizes: [256]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1071
   Correlation: 0.2641

5. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1079
   Correlation: 0.1123

Top 5 configurations 

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_hippocampus_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0929
   Correlation: 0.1766

2. Configuration:
   hidden_sizes: [128]
   dropout_rate: 0.5
   lr: 0.0005
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0958
   Correlation: 0.2266

3. Configuration:
   hidden_sizes: [64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1027
   Correlation: 0.1901

4. Configuration:
   hidden_sizes: [256]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1071
   Correlation: 0.2641

5. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1079
   Correlation: 0.1123

Top 5 configurations 

## Thalamus

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, randint
from scipy import linalg

# Load and preprocess data
def load_and_preprocess_data():
    densenet_df = pd.read_csv('densenet_thalamus_predictions_with_average_and_id.csv').sort_values('ID')
    pointnet_df = pd.read_csv('pointnet_thalamus_predictions_with_average_and_id.csv').sort_values('ID')
    random_forest_df = pd.read_csv('random_forest_pyradiomics_thalamus_predictions_with_average_and_id.csv').sort_values('ID')

    X = np.hstack((
        densenet_df.iloc[:, 2:17].values,
        pointnet_df.iloc[:, 2:17].values,
        random_forest_df.iloc[:, 2:17].values
    ))
    y = densenet_df['Actual'].values

    return X, y, densenet_df, pointnet_df, random_forest_df

# Ensemble methods
def simple_average_ensemble(X):
    return np.mean(X, axis=1)

def weighted_average_ensemble(X_train, y_train, X_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    return lr_model.predict(X_test)

def non_linear_weighted_average(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def tune_rf_meta_learner(X_train, y_train):
    param_dist = {
        'n_estimators': randint(50, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    }
    rf = RandomForestRegressor(random_state=42)
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)
    rf_random.fit(X_train, y_train)
    return rf_random.best_estimator_

def stacking_ensemble(X_train, y_train, X_test):
    models = [
        RandomForestRegressor(n_estimators=100, random_state=42),
        GradientBoostingRegressor(n_estimators=100, random_state=42),
        SVR(kernel='rbf')
    ]
    
    second_level_train = np.column_stack([
        cross_val_predict(model, X_train, y_train, cv=5) 
        for model in models
    ])
    second_level_test = np.column_stack([
        model.fit(X_train, y_train).predict(X_test) 
        for model in models
    ])
    
    # Use Ridge regression as the meta-learner
    alpha = 1.0
    n_samples, n_features = second_level_train.shape
    
    # Add a small value to the diagonal for numerical stability
    A = np.dot(second_level_train.T, second_level_train)
    A.flat[::n_features + 1] += alpha + 1e-10
    
    b = np.dot(second_level_train.T, y_train)
    
    # Solve the linear system
    try:
        coef = linalg.solve(A, b, assume_a='pos')
    except TypeError:
        # For older versions of SciPy
        coef = linalg.solve(A, b, sym_pos=True)
    
    # Make predictions
    return np.dot(second_level_test, coef)

# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    correlation, _ = pearsonr(y_true, y_pred)
    return mse, correlation

# Main execution
def main():
    X, y, densenet_df, pointnet_df, random_forest_df = load_and_preprocess_data()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    simple_avg_scores = []
    weighted_avg_scores = []
    non_linear_weighted_scores = []
    rf_meta_scores = []
    stacking_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Simple average
        simple_avg_pred = simple_average_ensemble(X_test)
        simple_avg_scores.append(evaluate_model(y_test, simple_avg_pred))
        
        # Weighted average
        weighted_avg_pred = weighted_average_ensemble(X_train, y_train, X_test)
        weighted_avg_scores.append(evaluate_model(y_test, weighted_avg_pred))
        
        # Non-linear weighted average
        non_linear_weighted_pred = non_linear_weighted_average(X_train, y_train, X_test)
        non_linear_weighted_scores.append(evaluate_model(y_test, non_linear_weighted_pred))
        
        # Random Forest meta-learner (with hyperparameter tuning)
        rf_model = tune_rf_meta_learner(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_meta_scores.append(evaluate_model(y_test, rf_pred))
        
        # Stacking ensemble
        stacking_pred = stacking_ensemble(X_train, y_train, X_test)
        stacking_scores.append(evaluate_model(y_test, stacking_pred))

    # Calculate and print average scores
    print("Ensemble Methods:")
    print(f"Simple Average Ensemble - MSE: {np.mean([s[0] for s in simple_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in simple_avg_scores]):.4f}")
    print(f"Weighted Average Ensemble - MSE: {np.mean([s[0] for s in weighted_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in weighted_avg_scores]):.4f}")
    print(f"Non-linear Weighted Average - MSE: {np.mean([s[0] for s in non_linear_weighted_scores]):.4f}, Correlation: {np.mean([s[1] for s in non_linear_weighted_scores]):.4f}")
    print(f"Tuned Random Forest Meta-learner - MSE: {np.mean([s[0] for s in rf_meta_scores]):.4f}, Correlation: {np.mean([s[1] for s in rf_meta_scores]):.4f}")
    print(f"Stacking Ensemble - MSE: {np.mean([s[0] for s in stacking_scores]):.4f}, Correlation: {np.mean([s[1] for s in stacking_scores]):.4f}")

    # Evaluate original models
    print("\nOriginal Models:")
    for model_name, df in zip(['DenseNet', 'PointNet', 'Random Forest'], 
                              [densenet_df, pointnet_df, random_forest_df]):
        avg_pred = df['Average_Prediction'].values
        mse, correlation = evaluate_model(df['Actual'], avg_pred)
        print(f"{model_name} - MSE: {mse:.4f}, Correlation: {correlation:.4f}")

if __name__ == "__main__":
    main()

Ensemble Methods:
Simple Average Ensemble - MSE: 0.0668, Correlation: 0.2280
Weighted Average Ensemble - MSE: 0.1196, Correlation: 0.3637
Non-linear Weighted Average - MSE: 0.0930, Correlation: 0.1776
Tuned Random Forest Meta-learner - MSE: 0.0767, Correlation: 0.0406
Stacking Ensemble - MSE: 0.0744, Correlation: 0.1081

Original Models:
DenseNet - MSE: 0.0724, Correlation: 0.2414
PointNet - MSE: 0.0760, Correlation: 0.1033
Random Forest - MSE: 0.0691, Correlation: 0.2111


### MLP with hyperparameters

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_thalamus_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_thalamus_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_thalamus_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and positive correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: x[2], reverse=True)

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Correlation
print_top_5(results_by_corr, "Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")

# Diagnostic: Print predictions vs actual for best correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")



Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0787
   Correlation: 0.2335

2. Configuration:
   hidden_sizes: [256]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0849
   Correlation: 0.2455

3. Configuration:
   hidden_sizes: [256]
   dropout_rate: 0.5
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0853
   Correlation: 0.3109

4. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0860
   Correlation: 0.2075

5. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0876
   Correlation: 0.2612

Top 5 configurations by 

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_thalamus_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_thalamus_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_thalamus_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0787
   Correlation: 0.2335

2. Configuration:
   hidden_sizes: [256]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0849
   Correlation: 0.2455

3. Configuration:
   hidden_sizes: [256]
   dropout_rate: 0.5
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0853
   Correlation: 0.3109

4. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0860
   Correlation: 0.2075

5. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0876
   Correlation: 0.2612

Top 5 configurations by 

## Whole brain

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, randint
from scipy import linalg

# Load and preprocess data
def load_and_preprocess_data():
    densenet_df = pd.read_csv('densenet_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')
    pointnet_df = pd.read_csv('pointnet_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')
    random_forest_df = pd.read_csv('random_forest_pyradiomics_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')

    X = np.hstack((
        densenet_df.iloc[:, 2:17].values,
        pointnet_df.iloc[:, 2:17].values,
        random_forest_df.iloc[:, 2:17].values
    ))
    y = densenet_df['Actual'].values

    return X, y, densenet_df, pointnet_df, random_forest_df

# Ensemble methods
def simple_average_ensemble(X):
    return np.mean(X, axis=1)

def weighted_average_ensemble(X_train, y_train, X_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    return lr_model.predict(X_test)

def non_linear_weighted_average(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def tune_rf_meta_learner(X_train, y_train):
    param_dist = {
        'n_estimators': randint(50, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    }
    rf = RandomForestRegressor(random_state=42)
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)
    rf_random.fit(X_train, y_train)
    return rf_random.best_estimator_

def stacking_ensemble(X_train, y_train, X_test):
    models = [
        RandomForestRegressor(n_estimators=100, random_state=42),
        GradientBoostingRegressor(n_estimators=100, random_state=42),
        SVR(kernel='rbf')
    ]
    
    second_level_train = np.column_stack([
        cross_val_predict(model, X_train, y_train, cv=5) 
        for model in models
    ])
    second_level_test = np.column_stack([
        model.fit(X_train, y_train).predict(X_test) 
        for model in models
    ])
    
    # Use Ridge regression as the meta-learner
    alpha = 1.0
    n_samples, n_features = second_level_train.shape
    
    # Add a small value to the diagonal for numerical stability
    A = np.dot(second_level_train.T, second_level_train)
    A.flat[::n_features + 1] += alpha + 1e-10
    
    b = np.dot(second_level_train.T, y_train)
    
    # Solve the linear system
    try:
        coef = linalg.solve(A, b, assume_a='pos')
    except TypeError:
        # For older versions of SciPy
        coef = linalg.solve(A, b, sym_pos=True)
    
    # Make predictions
    return np.dot(second_level_test, coef)

# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    correlation, _ = pearsonr(y_true, y_pred)
    return mse, correlation

# Main execution
def main():
    X, y, densenet_df, pointnet_df, random_forest_df = load_and_preprocess_data()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    simple_avg_scores = []
    weighted_avg_scores = []
    non_linear_weighted_scores = []
    rf_meta_scores = []
    stacking_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Simple average
        simple_avg_pred = simple_average_ensemble(X_test)
        simple_avg_scores.append(evaluate_model(y_test, simple_avg_pred))
        
        # Weighted average
        weighted_avg_pred = weighted_average_ensemble(X_train, y_train, X_test)
        weighted_avg_scores.append(evaluate_model(y_test, weighted_avg_pred))
        
        # Non-linear weighted average
        non_linear_weighted_pred = non_linear_weighted_average(X_train, y_train, X_test)
        non_linear_weighted_scores.append(evaluate_model(y_test, non_linear_weighted_pred))
        
        # Random Forest meta-learner (with hyperparameter tuning)
        rf_model = tune_rf_meta_learner(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_meta_scores.append(evaluate_model(y_test, rf_pred))
        
        # Stacking ensemble
        stacking_pred = stacking_ensemble(X_train, y_train, X_test)
        stacking_scores.append(evaluate_model(y_test, stacking_pred))

    # Calculate and print average scores
    print("Ensemble Methods:")
    print(f"Simple Average Ensemble - MSE: {np.mean([s[0] for s in simple_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in simple_avg_scores]):.4f}")
    print(f"Weighted Average Ensemble - MSE: {np.mean([s[0] for s in weighted_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in weighted_avg_scores]):.4f}")
    print(f"Non-linear Weighted Average - MSE: {np.mean([s[0] for s in non_linear_weighted_scores]):.4f}, Correlation: {np.mean([s[1] for s in non_linear_weighted_scores]):.4f}")
    print(f"Tuned Random Forest Meta-learner - MSE: {np.mean([s[0] for s in rf_meta_scores]):.4f}, Correlation: {np.mean([s[1] for s in rf_meta_scores]):.4f}")
    print(f"Stacking Ensemble - MSE: {np.mean([s[0] for s in stacking_scores]):.4f}, Correlation: {np.mean([s[1] for s in stacking_scores]):.4f}")

    # Evaluate original models
    print("\nOriginal Models:")
    for model_name, df in zip(['DenseNet', 'PointNet', 'Random Forest'], 
                              [densenet_df, pointnet_df, random_forest_df]):
        avg_pred = df['Average_Prediction'].values
        mse, correlation = evaluate_model(df['Actual'], avg_pred)
        print(f"{model_name} - MSE: {mse:.4f}, Correlation: {correlation:.4f}")

if __name__ == "__main__":
    main()

Ensemble Methods:
Simple Average Ensemble - MSE: 0.0711, Correlation: 0.1928
Weighted Average Ensemble - MSE: 0.2491, Correlation: 0.0845
Non-linear Weighted Average - MSE: 0.1149, Correlation: 0.1087
Tuned Random Forest Meta-learner - MSE: 0.0799, Correlation: 0.0349
Stacking Ensemble - MSE: 0.0833, Correlation: 0.0256

Original Models:
DenseNet - MSE: 0.0835, Correlation: 0.1025
PointNet - MSE: 0.0749, Correlation: 0.1843
Random Forest - MSE: 0.0678, Correlation: 0.2474


### MLP with hyperparameters

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and positive correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: x[2], reverse=True)

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Correlation
print_top_5(results_by_corr, "Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")

# Diagnostic: Print predictions vs actual for best correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")



Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0897
   Correlation: 0.2398

2. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0947
   Correlation: 0.2438

3. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0957
   Correlation: 0.1730

4. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0981
   Correlation: 0.2876

5. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1007
   Correlation: 0.1699

To

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_wholebrain_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0897
   Correlation: 0.2398

2. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0947
   Correlation: 0.2438

3. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0957
   Correlation: 0.1730

4. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0981
   Correlation: 0.2876

5. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1007
   Correlation: 0.1699

To

## Amygdala

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, randint
from scipy import linalg

# Load and preprocess data
def load_and_preprocess_data():
    densenet_df = pd.read_csv('densenet_amygdala_predictions_with_average_and_id.csv').sort_values('ID')
    pointnet_df = pd.read_csv('pointnet_amygdala_predictions_with_average_and_id.csv').sort_values('ID')
    random_forest_df = pd.read_csv('random_forest_pyradiomics_amygdala_predictions_with_average_and_id.csv').sort_values('ID')

    X = np.hstack((
        densenet_df.iloc[:, 2:17].values,
        pointnet_df.iloc[:, 2:17].values,
        random_forest_df.iloc[:, 2:17].values
    ))
    y = densenet_df['Actual'].values

    return X, y, densenet_df, pointnet_df, random_forest_df

# Ensemble methods
def simple_average_ensemble(X):
    return np.mean(X, axis=1)

def weighted_average_ensemble(X_train, y_train, X_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    return lr_model.predict(X_test)

def non_linear_weighted_average(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def tune_rf_meta_learner(X_train, y_train):
    param_dist = {
        'n_estimators': randint(50, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    }
    rf = RandomForestRegressor(random_state=42)
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)
    rf_random.fit(X_train, y_train)
    return rf_random.best_estimator_

def stacking_ensemble(X_train, y_train, X_test):
    models = [
        RandomForestRegressor(n_estimators=100, random_state=42),
        GradientBoostingRegressor(n_estimators=100, random_state=42),
        SVR(kernel='rbf')
    ]
    
    second_level_train = np.column_stack([
        cross_val_predict(model, X_train, y_train, cv=5) 
        for model in models
    ])
    second_level_test = np.column_stack([
        model.fit(X_train, y_train).predict(X_test) 
        for model in models
    ])
    
    # Use Ridge regression as the meta-learner
    alpha = 1.0
    n_samples, n_features = second_level_train.shape
    
    # Add a small value to the diagonal for numerical stability
    A = np.dot(second_level_train.T, second_level_train)
    A.flat[::n_features + 1] += alpha + 1e-10
    
    b = np.dot(second_level_train.T, y_train)
    
    # Solve the linear system
    try:
        coef = linalg.solve(A, b, assume_a='pos')
    except TypeError:
        # For older versions of SciPy
        coef = linalg.solve(A, b, sym_pos=True)
    
    # Make predictions
    return np.dot(second_level_test, coef)

# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    correlation, _ = pearsonr(y_true, y_pred)
    return mse, correlation

# Main execution
def main():
    X, y, densenet_df, pointnet_df, random_forest_df = load_and_preprocess_data()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    simple_avg_scores = []
    weighted_avg_scores = []
    non_linear_weighted_scores = []
    rf_meta_scores = []
    stacking_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Simple average
        simple_avg_pred = simple_average_ensemble(X_test)
        simple_avg_scores.append(evaluate_model(y_test, simple_avg_pred))
        
        # Weighted average
        weighted_avg_pred = weighted_average_ensemble(X_train, y_train, X_test)
        weighted_avg_scores.append(evaluate_model(y_test, weighted_avg_pred))
        
        # Non-linear weighted average
        non_linear_weighted_pred = non_linear_weighted_average(X_train, y_train, X_test)
        non_linear_weighted_scores.append(evaluate_model(y_test, non_linear_weighted_pred))
        
        # Random Forest meta-learner (with hyperparameter tuning)
        rf_model = tune_rf_meta_learner(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_meta_scores.append(evaluate_model(y_test, rf_pred))
        
        # Stacking ensemble
        stacking_pred = stacking_ensemble(X_train, y_train, X_test)
        stacking_scores.append(evaluate_model(y_test, stacking_pred))

    # Calculate and print average scores
    print("Ensemble Methods:")
    print(f"Simple Average Ensemble - MSE: {np.mean([s[0] for s in simple_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in simple_avg_scores]):.4f}")
    print(f"Weighted Average Ensemble - MSE: {np.mean([s[0] for s in weighted_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in weighted_avg_scores]):.4f}")
    print(f"Non-linear Weighted Average - MSE: {np.mean([s[0] for s in non_linear_weighted_scores]):.4f}, Correlation: {np.mean([s[1] for s in non_linear_weighted_scores]):.4f}")
    print(f"Tuned Random Forest Meta-learner - MSE: {np.mean([s[0] for s in rf_meta_scores]):.4f}, Correlation: {np.mean([s[1] for s in rf_meta_scores]):.4f}")
    print(f"Stacking Ensemble - MSE: {np.mean([s[0] for s in stacking_scores]):.4f}, Correlation: {np.mean([s[1] for s in stacking_scores]):.4f}")

    # Evaluate original models
    print("\nOriginal Models:")
    for model_name, df in zip(['DenseNet', 'PointNet', 'Random Forest'], 
                              [densenet_df, pointnet_df, random_forest_df]):
        avg_pred = df['Average_Prediction'].values
        mse, correlation = evaluate_model(df['Actual'], avg_pred)
        print(f"{model_name} - MSE: {mse:.4f}, Correlation: {correlation:.4f}")

if __name__ == "__main__":
    main()

Ensemble Methods:
Simple Average Ensemble - MSE: 0.0709, Correlation: 0.1383
Weighted Average Ensemble - MSE: 0.1594, Correlation: 0.1758
Non-linear Weighted Average - MSE: 0.0980, Correlation: 0.0715
Tuned Random Forest Meta-learner - MSE: 0.0751, Correlation: 0.0333
Stacking Ensemble - MSE: 0.0724, Correlation: 0.1399

Original Models:
DenseNet - MSE: 0.0779, Correlation: 0.1081
PointNet - MSE: 0.0776, Correlation: 0.0420
Random Forest - MSE: 0.0694, Correlation: 0.1986


### MLP with hyperparameters

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_amygdala_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_amygdala_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_amygdala_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and positive correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: x[2], reverse=True)

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Correlation
print_top_5(results_by_corr, "Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")

# Diagnostic: Print predictions vs actual for best correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")



Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0904
   Correlation: 0.2194

2. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0922
   Correlation: 0.2261

3. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0956
   Correlation: 0.1369

4. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0969
   Correlation: 0.1986

5. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0983
   Correlation: 0.2232

Top 5 configurati

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_amygdala_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_amygdala_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_amygdala_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0904
   Correlation: 0.2194

2. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0922
   Correlation: 0.2261

3. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0956
   Correlation: 0.1369

4. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0969
   Correlation: 0.1986

5. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0983
   Correlation: 0.2232

Top 5 configurati

## Isthmus cingulate

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, randint
from scipy import linalg

# Load and preprocess data
def load_and_preprocess_data():
    densenet_df = pd.read_csv('densenet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
    pointnet_df = pd.read_csv('pointnet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
    random_forest_df = pd.read_csv('random_forest_pyradiomics_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')

    X = np.hstack((
        densenet_df.iloc[:, 2:17].values,
        pointnet_df.iloc[:, 2:17].values,
        random_forest_df.iloc[:, 2:17].values
    ))
    y = densenet_df['Actual'].values

    return X, y, densenet_df, pointnet_df, random_forest_df

# Ensemble methods
def simple_average_ensemble(X):
    return np.mean(X, axis=1)

def weighted_average_ensemble(X_train, y_train, X_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    return lr_model.predict(X_test)

def non_linear_weighted_average(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def tune_rf_meta_learner(X_train, y_train):
    param_dist = {
        'n_estimators': randint(50, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    }
    rf = RandomForestRegressor(random_state=42)
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)
    rf_random.fit(X_train, y_train)
    return rf_random.best_estimator_

def stacking_ensemble(X_train, y_train, X_test):
    models = [
        RandomForestRegressor(n_estimators=100, random_state=42),
        GradientBoostingRegressor(n_estimators=100, random_state=42),
        SVR(kernel='rbf')
    ]
    
    second_level_train = np.column_stack([
        cross_val_predict(model, X_train, y_train, cv=5) 
        for model in models
    ])
    second_level_test = np.column_stack([
        model.fit(X_train, y_train).predict(X_test) 
        for model in models
    ])
    
    # Use Ridge regression as the meta-learner
    alpha = 1.0
    n_samples, n_features = second_level_train.shape
    
    # Add a small value to the diagonal for numerical stability
    A = np.dot(second_level_train.T, second_level_train)
    A.flat[::n_features + 1] += alpha + 1e-10
    
    b = np.dot(second_level_train.T, y_train)
    
    # Solve the linear system
    try:
        coef = linalg.solve(A, b, assume_a='pos')
    except TypeError:
        # For older versions of SciPy
        coef = linalg.solve(A, b, sym_pos=True)
    
    # Make predictions
    return np.dot(second_level_test, coef)

# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    correlation, _ = pearsonr(y_true, y_pred)
    return mse, correlation

# Main execution
def main():
    X, y, densenet_df, pointnet_df, random_forest_df = load_and_preprocess_data()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    simple_avg_scores = []
    weighted_avg_scores = []
    non_linear_weighted_scores = []
    rf_meta_scores = []
    stacking_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Simple average
        simple_avg_pred = simple_average_ensemble(X_test)
        simple_avg_scores.append(evaluate_model(y_test, simple_avg_pred))
        
        # Weighted average
        weighted_avg_pred = weighted_average_ensemble(X_train, y_train, X_test)
        weighted_avg_scores.append(evaluate_model(y_test, weighted_avg_pred))
        
        # Non-linear weighted average
        non_linear_weighted_pred = non_linear_weighted_average(X_train, y_train, X_test)
        non_linear_weighted_scores.append(evaluate_model(y_test, non_linear_weighted_pred))
        
        # Random Forest meta-learner (with hyperparameter tuning)
        rf_model = tune_rf_meta_learner(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_meta_scores.append(evaluate_model(y_test, rf_pred))
        
        # Stacking ensemble
        stacking_pred = stacking_ensemble(X_train, y_train, X_test)
        stacking_scores.append(evaluate_model(y_test, stacking_pred))

    # Calculate and print average scores
    print("Ensemble Methods:")
    print(f"Simple Average Ensemble - MSE: {np.mean([s[0] for s in simple_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in simple_avg_scores]):.4f}")
    print(f"Weighted Average Ensemble - MSE: {np.mean([s[0] for s in weighted_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in weighted_avg_scores]):.4f}")
    print(f"Non-linear Weighted Average - MSE: {np.mean([s[0] for s in non_linear_weighted_scores]):.4f}, Correlation: {np.mean([s[1] for s in non_linear_weighted_scores]):.4f}")
    print(f"Tuned Random Forest Meta-learner - MSE: {np.mean([s[0] for s in rf_meta_scores]):.4f}, Correlation: {np.mean([s[1] for s in rf_meta_scores]):.4f}")
    print(f"Stacking Ensemble - MSE: {np.mean([s[0] for s in stacking_scores]):.4f}, Correlation: {np.mean([s[1] for s in stacking_scores]):.4f}")

    # Evaluate original models
    print("\nOriginal Models:")
    for model_name, df in zip(['DenseNet', 'PointNet', 'Random Forest'], 
                              [densenet_df, pointnet_df, random_forest_df]):
        avg_pred = df['Average_Prediction'].values
        mse, correlation = evaluate_model(df['Actual'], avg_pred)
        print(f"{model_name} - MSE: {mse:.4f}, Correlation: {correlation:.4f}")

if __name__ == "__main__":
    main()

Ensemble Methods:
Simple Average Ensemble - MSE: 0.0708, Correlation: 0.0820
Weighted Average Ensemble - MSE: 0.2867, Correlation: -0.1368
Non-linear Weighted Average - MSE: 0.1210, Correlation: 0.0096
Tuned Random Forest Meta-learner - MSE: 0.0791, Correlation: -0.0689
Stacking Ensemble - MSE: 0.0819, Correlation: -0.0554

Original Models:
DenseNet - MSE: 0.0731, Correlation: 0.1591
PointNet - MSE: 0.0846, Correlation: -0.0114
Random Forest - MSE: 0.0695, Correlation: 0.1990


### MLP with hyperparameters

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and positive correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: x[2], reverse=True)

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Correlation
print_top_5(results_by_corr, "Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")

# Diagnostic: Print predictions vs actual for best correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")



Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0980
   Correlation: 0.1363

2. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1071
   Correlation: -0.0241

3. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1074
   Correlation: 0.0023

4. Configuration:
   hidden_sizes: [128]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1091
   Correlation: 0.1292

5. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1093
   Correlation: 0.0380

Top 5 

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0980
   Correlation: 0.1363

2. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1071
   Correlation: -0.0241

3. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1074
   Correlation: 0.0023

4. Configuration:
   hidden_sizes: [128]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1091
   Correlation: 0.1292

5. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1093
   Correlation: 0.0380

Top 5 

## Parahippocampus

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, randint
from scipy import linalg

# Load and preprocess data
def load_and_preprocess_data():
    densenet_df = pd.read_csv('densenet_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')
    pointnet_df = pd.read_csv('pointnet_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')
    random_forest_df = pd.read_csv('random_forest_pyradiomics_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')

    X = np.hstack((
        densenet_df.iloc[:, 2:17].values,
        pointnet_df.iloc[:, 2:17].values,
        random_forest_df.iloc[:, 2:17].values
    ))
    y = densenet_df['Actual'].values

    return X, y, densenet_df, pointnet_df, random_forest_df

# Ensemble methods
def simple_average_ensemble(X):
    return np.mean(X, axis=1)

def weighted_average_ensemble(X_train, y_train, X_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    return lr_model.predict(X_test)

def non_linear_weighted_average(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def tune_rf_meta_learner(X_train, y_train):
    param_dist = {
        'n_estimators': randint(50, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    }
    rf = RandomForestRegressor(random_state=42)
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)
    rf_random.fit(X_train, y_train)
    return rf_random.best_estimator_

def stacking_ensemble(X_train, y_train, X_test):
    models = [
        RandomForestRegressor(n_estimators=100, random_state=42),
        GradientBoostingRegressor(n_estimators=100, random_state=42),
        SVR(kernel='rbf')
    ]
    
    second_level_train = np.column_stack([
        cross_val_predict(model, X_train, y_train, cv=5) 
        for model in models
    ])
    second_level_test = np.column_stack([
        model.fit(X_train, y_train).predict(X_test) 
        for model in models
    ])
    
    # Use Ridge regression as the meta-learner
    alpha = 1.0
    n_samples, n_features = second_level_train.shape
    
    # Add a small value to the diagonal for numerical stability
    A = np.dot(second_level_train.T, second_level_train)
    A.flat[::n_features + 1] += alpha + 1e-10
    
    b = np.dot(second_level_train.T, y_train)
    
    # Solve the linear system
    try:
        coef = linalg.solve(A, b, assume_a='pos')
    except TypeError:
        # For older versions of SciPy
        coef = linalg.solve(A, b, sym_pos=True)
    
    # Make predictions
    return np.dot(second_level_test, coef)

# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    correlation, _ = pearsonr(y_true, y_pred)
    return mse, correlation

# Main execution
def main():
    X, y, densenet_df, pointnet_df, random_forest_df = load_and_preprocess_data()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    simple_avg_scores = []
    weighted_avg_scores = []
    non_linear_weighted_scores = []
    rf_meta_scores = []
    stacking_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Simple average
        simple_avg_pred = simple_average_ensemble(X_test)
        simple_avg_scores.append(evaluate_model(y_test, simple_avg_pred))
        
        # Weighted average
        weighted_avg_pred = weighted_average_ensemble(X_train, y_train, X_test)
        weighted_avg_scores.append(evaluate_model(y_test, weighted_avg_pred))
        
        # Non-linear weighted average
        non_linear_weighted_pred = non_linear_weighted_average(X_train, y_train, X_test)
        non_linear_weighted_scores.append(evaluate_model(y_test, non_linear_weighted_pred))
        
        # Random Forest meta-learner (with hyperparameter tuning)
        rf_model = tune_rf_meta_learner(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_meta_scores.append(evaluate_model(y_test, rf_pred))
        
        # Stacking ensemble
        stacking_pred = stacking_ensemble(X_train, y_train, X_test)
        stacking_scores.append(evaluate_model(y_test, stacking_pred))

    # Calculate and print average scores
    print("Ensemble Methods:")
    print(f"Simple Average Ensemble - MSE: {np.mean([s[0] for s in simple_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in simple_avg_scores]):.4f}")
    print(f"Weighted Average Ensemble - MSE: {np.mean([s[0] for s in weighted_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in weighted_avg_scores]):.4f}")
    print(f"Non-linear Weighted Average - MSE: {np.mean([s[0] for s in non_linear_weighted_scores]):.4f}, Correlation: {np.mean([s[1] for s in non_linear_weighted_scores]):.4f}")
    print(f"Tuned Random Forest Meta-learner - MSE: {np.mean([s[0] for s in rf_meta_scores]):.4f}, Correlation: {np.mean([s[1] for s in rf_meta_scores]):.4f}")
    print(f"Stacking Ensemble - MSE: {np.mean([s[0] for s in stacking_scores]):.4f}, Correlation: {np.mean([s[1] for s in stacking_scores]):.4f}")

    # Evaluate original models
    print("\nOriginal Models:")
    for model_name, df in zip(['DenseNet', 'PointNet', 'Random Forest'], 
                              [densenet_df, pointnet_df, random_forest_df]):
        avg_pred = df['Average_Prediction'].values
        mse, correlation = evaluate_model(df['Actual'], avg_pred)
        print(f"{model_name} - MSE: {mse:.4f}, Correlation: {correlation:.4f}")

if __name__ == "__main__":
    main()

Ensemble Methods:
Simple Average Ensemble - MSE: 0.0720, Correlation: 0.0819
Weighted Average Ensemble - MSE: 0.2188, Correlation: -0.0072
Non-linear Weighted Average - MSE: 0.0955, Correlation: 0.2044
Tuned Random Forest Meta-learner - MSE: 0.0754, Correlation: 0.1011
Stacking Ensemble - MSE: 0.0739, Correlation: 0.1503

Original Models:
DenseNet - MSE: 0.0755, Correlation: 0.1722
PointNet - MSE: 0.0862, Correlation: -0.0811
Random Forest - MSE: 0.0688, Correlation: 0.2166


### MLP with hyperparameters

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and positive correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: x[2], reverse=True)

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Correlation
print_top_5(results_by_corr, "Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")

# Diagnostic: Print predictions vs actual for best correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")



Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0846
   Correlation: 0.3092

2. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0917
   Correlation: 0.1818

3. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0932
   Correlation: 0.3336

4. Configuration:
   hidden_sizes: [64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0952
   Correlation: 0.1974

5. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1015
   Correlation: 0.1923

Top 

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_parahippocampus_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0846
   Correlation: 0.3092

2. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0917
   Correlation: 0.1818

3. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0932
   Correlation: 0.3336

4. Configuration:
   hidden_sizes: [64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0952
   Correlation: 0.1974

5. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1015
   Correlation: 0.1923

Top 

## Entorhinal

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, randint
from scipy import linalg

# Load and preprocess data
def load_and_preprocess_data():
    densenet_df = pd.read_csv('densenet_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')
    pointnet_df = pd.read_csv('pointnet_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')
    random_forest_df = pd.read_csv('random_forest_pyradiomics_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')

    X = np.hstack((
        densenet_df.iloc[:, 2:17].values,
        pointnet_df.iloc[:, 2:17].values,
        random_forest_df.iloc[:, 2:17].values
    ))
    y = densenet_df['Actual'].values

    return X, y, densenet_df, pointnet_df, random_forest_df

# Ensemble methods
def simple_average_ensemble(X):
    return np.mean(X, axis=1)

def weighted_average_ensemble(X_train, y_train, X_test):
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    return lr_model.predict(X_test)

def non_linear_weighted_average(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def tune_rf_meta_learner(X_train, y_train):
    param_dist = {
        'n_estimators': randint(50, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    }
    rf = RandomForestRegressor(random_state=42)
    rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)
    rf_random.fit(X_train, y_train)
    return rf_random.best_estimator_

def stacking_ensemble(X_train, y_train, X_test):
    models = [
        RandomForestRegressor(n_estimators=100, random_state=42),
        GradientBoostingRegressor(n_estimators=100, random_state=42),
        SVR(kernel='rbf')
    ]
    
    second_level_train = np.column_stack([
        cross_val_predict(model, X_train, y_train, cv=5) 
        for model in models
    ])
    second_level_test = np.column_stack([
        model.fit(X_train, y_train).predict(X_test) 
        for model in models
    ])
    
    # Use Ridge regression as the meta-learner
    alpha = 1.0
    n_samples, n_features = second_level_train.shape
    
    # Add a small value to the diagonal for numerical stability
    A = np.dot(second_level_train.T, second_level_train)
    A.flat[::n_features + 1] += alpha + 1e-10
    
    b = np.dot(second_level_train.T, y_train)
    
    # Solve the linear system
    try:
        coef = linalg.solve(A, b, assume_a='pos')
    except TypeError:
        # For older versions of SciPy
        coef = linalg.solve(A, b, sym_pos=True)
    
    # Make predictions
    return np.dot(second_level_test, coef)

# Evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    correlation, _ = pearsonr(y_true, y_pred)
    return mse, correlation

# Main execution
def main():
    X, y, densenet_df, pointnet_df, random_forest_df = load_and_preprocess_data()

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    simple_avg_scores = []
    weighted_avg_scores = []
    non_linear_weighted_scores = []
    rf_meta_scores = []
    stacking_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Simple average
        simple_avg_pred = simple_average_ensemble(X_test)
        simple_avg_scores.append(evaluate_model(y_test, simple_avg_pred))
        
        # Weighted average
        weighted_avg_pred = weighted_average_ensemble(X_train, y_train, X_test)
        weighted_avg_scores.append(evaluate_model(y_test, weighted_avg_pred))
        
        # Non-linear weighted average
        non_linear_weighted_pred = non_linear_weighted_average(X_train, y_train, X_test)
        non_linear_weighted_scores.append(evaluate_model(y_test, non_linear_weighted_pred))
        
        # Random Forest meta-learner (with hyperparameter tuning)
        rf_model = tune_rf_meta_learner(X_train, y_train)
        rf_pred = rf_model.predict(X_test)
        rf_meta_scores.append(evaluate_model(y_test, rf_pred))
        
        # Stacking ensemble
        stacking_pred = stacking_ensemble(X_train, y_train, X_test)
        stacking_scores.append(evaluate_model(y_test, stacking_pred))

    # Calculate and print average scores
    print("Ensemble Methods:")
    print(f"Simple Average Ensemble - MSE: {np.mean([s[0] for s in simple_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in simple_avg_scores]):.4f}")
    print(f"Weighted Average Ensemble - MSE: {np.mean([s[0] for s in weighted_avg_scores]):.4f}, Correlation: {np.mean([s[1] for s in weighted_avg_scores]):.4f}")
    print(f"Non-linear Weighted Average - MSE: {np.mean([s[0] for s in non_linear_weighted_scores]):.4f}, Correlation: {np.mean([s[1] for s in non_linear_weighted_scores]):.4f}")
    print(f"Tuned Random Forest Meta-learner - MSE: {np.mean([s[0] for s in rf_meta_scores]):.4f}, Correlation: {np.mean([s[1] for s in rf_meta_scores]):.4f}")
    print(f"Stacking Ensemble - MSE: {np.mean([s[0] for s in stacking_scores]):.4f}, Correlation: {np.mean([s[1] for s in stacking_scores]):.4f}")

    # Evaluate original models
    print("\nOriginal Models:")
    for model_name, df in zip(['DenseNet', 'PointNet', 'Random Forest'], 
                              [densenet_df, pointnet_df, random_forest_df]):
        avg_pred = df['Average_Prediction'].values
        mse, correlation = evaluate_model(df['Actual'], avg_pred)
        print(f"{model_name} - MSE: {mse:.4f}, Correlation: {correlation:.4f}")

if __name__ == "__main__":
    main()

Ensemble Methods:
Simple Average Ensemble - MSE: 0.0761, Correlation: -0.0941
Weighted Average Ensemble - MSE: 0.2303, Correlation: -0.0990
Non-linear Weighted Average - MSE: 0.1107, Correlation: 0.0444
Tuned Random Forest Meta-learner - MSE: 0.0711, Correlation: 0.1850
Stacking Ensemble - MSE: 0.0738, Correlation: 0.1365

Original Models:
DenseNet - MSE: 0.0841, Correlation: -0.0657
PointNet - MSE: 0.0828, Correlation: -0.1635
Random Forest - MSE: 0.0714, Correlation: 0.1222


### MLP with hyperparameters

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and positive correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: x[2], reverse=True)

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Correlation
print_top_5(results_by_corr, "Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")

# Diagnostic: Print predictions vs actual for best correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")



Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0926
   Correlation: 0.1728

2. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0948
   Correlation: 0.0486

3. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0973
   Correlation: 0.1017

4. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1015
   Correlation: 0.1772

5. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1031
   Correlation: 0.0477

Top 5 configu

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_entorhinal_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [64]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0926
   Correlation: 0.1728

2. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0948
   Correlation: 0.0486

3. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0973
   Correlation: 0.1017

4. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1015
   Correlation: 0.1772

5. Configuration:
   hidden_sizes: [256, 128]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1031
   Correlation: 0.0477

Top 5 configu

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from itertools import product
import random

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a global seed
GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

class MLPEnsemble(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLPEnsemble, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def train_model(model, X_train, y_train, X_val, y_val, epochs, patience, lr, weight_decay, seed):
    set_seed(seed)  # Set seed for this training run
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.squeeze(), y_val)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            break
    
    return model

def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze()
    
    y_np = y.numpy() if isinstance(y, torch.Tensor) else y
    predictions_np = predictions.numpy() if isinstance(predictions, torch.Tensor) else predictions
    
    mse = mean_squared_error(y_np, predictions_np)
    correlation, _ = pearsonr(y_np, predictions_np)
    return mse, correlation

def cross_validate_mlp(X, y, n_splits=5, **params):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=GLOBAL_SEED)
    mse_scores = []
    corr_scores = []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model_params = {
        'input_size': X.shape[1],
        'hidden_sizes': params['hidden_sizes'],
        'output_size': params['output_size'],
        'dropout_rate': params['dropout_rate']
    }
    train_params = {
        'epochs': params['epochs'],
        'patience': params['patience'],
        'lr': params['lr'],
        'weight_decay': params['weight_decay'],
        'seed': GLOBAL_SEED
    }
    
    for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.FloatTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.FloatTensor(y_val)
        
        model = MLPEnsemble(**model_params)
        model = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, **train_params)
        
        mse, corr = evaluate_model(model, X_val_tensor, y_val_tensor)
        mse_scores.append(mse)
        corr_scores.append(corr)
        
        #print(f"Fold {fold+1} - MSE: {mse:.4f}, Correlation: {corr:.4f}")
    
    return np.mean(mse_scores), np.mean(corr_scores)

# Load and preprocess data
densenet_df = pd.read_csv('densenet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
pointnet_df = pd.read_csv('pointnet_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')
random_forest_df = pd.read_csv('random_forest_pyradiomics_isthmuscingulate_predictions_with_average_and_id.csv').sort_values('ID')

X = np.hstack((
    densenet_df.iloc[:, 2:17].values,
    pointnet_df.iloc[:, 2:17].values,
    random_forest_df.iloc[:, 2:17].values
))
y = densenet_df['Actual'].values

# Define hyperparameter grid
param_grid = {
    'hidden_sizes': [[64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32]],
    'dropout_rate': [0, 0.3, 0.5],
    'lr': [0.001, 0.0005, 0.0001],
    'weight_decay': [1e-4, 1e-5],
    'epochs': [1000],
    'patience': [50],
    'output_size': [1]
}

# Perform grid search
results = []
for params in product(*param_grid.values()):
    config = dict(zip(param_grid.keys(), params))
    mse, corr = cross_validate_mlp(X, y, **config)
    results.append((config, mse, corr))
    #print(f"Config: {config}")
    #print(f"MSE: {mse:.4f}, Correlation: {corr:.4f}")
    #print("-" * 50)

# Sort results by MSE (ascending) and absolute correlation (descending)
results_by_mse = sorted(results, key=lambda x: x[1])
results_by_corr = sorted(results, key=lambda x: abs(x[2]), reverse=True)  # Modified to use absolute value

def print_top_5(sorted_results, metric_name):
    print(f"\nTop 5 configurations by {metric_name}:")
    for i, (config, mse, corr) in enumerate(sorted_results[:5], 1):
        print(f"\n{i}. Configuration:")
        for key, value in config.items():
            print(f"   {key}: {value}")
        print(f"   MSE: {mse:.4f}")
        print(f"   Correlation: {corr:.4f}")
        if metric_name == "Absolute Correlation":  # Show absolute value for clarity
            print(f"   |Correlation|: {abs(corr):.4f}")

# Print top 5 by MSE
print_top_5(results_by_mse, "MSE")

# Print top 5 by Absolute Correlation
print_top_5(results_by_corr, "Absolute Correlation")

# Best configuration by MSE
best_config_mse, best_mse, _ = results_by_mse[0]
print("\nBest configuration by MSE:")
for key, value in best_config_mse.items():
    print(f"{key}: {value}")
print(f"MSE: {best_mse:.4f}")

# Best configuration by Absolute Correlation
best_config_corr, _, best_corr = results_by_corr[0]
print("\nBest configuration by Absolute Correlation:")
for key, value in best_config_corr.items():
    print(f"{key}: {value}")
print(f"Correlation: {best_corr:.4f}")
print(f"|Correlation|: {abs(best_corr):.4f}")

# Diagnostic: Print predictions vs actual for best absolute correlation model
input_size = X.shape[1]
best_model = MLPEnsemble(
    input_size=input_size,
    hidden_sizes=best_config_corr['hidden_sizes'],
    output_size=best_config_corr['output_size'],
    dropout_rate=best_config_corr['dropout_rate']
)
best_model = train_model(best_model, torch.FloatTensor(X), torch.FloatTensor(y), torch.FloatTensor(X), torch.FloatTensor(y), 
                         epochs=best_config_corr['epochs'], patience=best_config_corr['patience'], 
                         lr=best_config_corr['lr'], weight_decay=best_config_corr['weight_decay'], seed=GLOBAL_SEED)

with torch.no_grad():
    predictions = best_model(torch.FloatTensor(X)).squeeze().numpy()
    
# print("\nSample of predictions vs actual:")
# for i in range(min(20, len(y))):
#     print(f"Actual: {y[i]:.4f}, Predicted: {predictions[i]:.4f}")

# # Correlation of individual input features with target
# print("\nCorrelation of individual features with target:")
# for i in range(X.shape[1]):
#     corr, _ = pearsonr(X[:, i], y)
#     print(f"Feature {i+1}: {corr:.4f}")


Top 5 configurations by MSE:

1. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.5
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.0980
   Correlation: 0.1363

2. Configuration:
   hidden_sizes: [128, 64, 32]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1071
   Correlation: -0.0241

3. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1074
   Correlation: 0.0023

4. Configuration:
   hidden_sizes: [128]
   dropout_rate: 0.3
   lr: 0.0005
   weight_decay: 1e-05
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1091
   Correlation: 0.1292

5. Configuration:
   hidden_sizes: [128, 64]
   dropout_rate: 0.3
   lr: 0.001
   weight_decay: 0.0001
   epochs: 1000
   patience: 50
   output_size: 1
   MSE: 0.1093
   Correlation: 0.0380

Top 5 